In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
!ls "/content/drive/MyDrive/CiC-DataSet"
!ls "/content/drive/MyDrive/CiC-DataSet/Complete_Dataset"
!ls "/content/drive/MyDrive/CiC-DataSet/Complete_Dataset/csv"

Complete_Dataset  merged_filtered.csv
 csv   example	'README - README.pdf'   supplementary
 CICIoT2023  'README_csv - README.pdf'


In [ ]:
import pandas as pd
import numpy as np
import os
from tqdm import tqdm
import warnings
from sklearn.ensemble import RandomForestClassifier
from google.colab import drive
from sklearn.preprocessing import MinMaxScaler, StandardScaler

# Forcibly Mount Google Drive
drive.mount('/content/drive', force_remount=True)

# Define Dataset Directory
DATASET_DIRECTORY = '/content/drive/MyDrive/CiC-DataSet/Complete_Dataset/csv/CICIoT2023'

# Verify if the directory exists and list CSV files
try:
    df_sets = [k for k in os.listdir(DATASET_DIRECTORY) if k.endswith('.csv')]
    if not df_sets:
        print("No CSV files found in the specified directory.")
    df_sets.sort()
except Exception as e:
    print(f"Error accessing directory: {e}")

# Split into training and test sets
training_sets = df_sets[:int(len(df_sets) * 0.8)]
test_sets = df_sets[int(len(df_sets) * 0.8):]

# Define columns
X_columns = [
    'flow_duration', 'Header_Length', 'Protocol Type', 'Duration',
    'Rate', 'Srate', 'Drate', 'fin_flag_number', 'syn_flag_number',
    'rst_flag_number', 'psh_flag_number', 'ack_flag_number',
    'ece_flag_number', 'cwr_flag_number', 'ack_count',
    'syn_count', 'fin_count', 'urg_count', 'rst_count',
    'HTTP', 'HTTPS', 'DNS', 'Telnet', 'SMTP', 'SSH', 'IRC', 'TCP',
    'UDP', 'DHCP', 'ARP', 'ICMP', 'IPv', 'LLC', 'Tot sum', 'Min',
    'Max', 'AVG', 'Std', 'Tot size', 'IAT', 'Number', 'Magnitue',
    'Radius', 'Covariance', 'Variance', 'Weight',
]
y_column = 'label'

# Initialize the scaler
scaler = StandardScaler()

# Fit scaler to training sets with error handling
for train_set in tqdm(training_sets, desc="Processing Training Sets"):
    file_path = os.path.join(DATASET_DIRECTORY, train_set)
    try:
        for chunk in pd.read_csv(file_path, usecols=X_columns, chunksize=5000):  # Reduced chunksize to 5000
            scaler.partial_fit(chunk)
    except Exception as e:
        print(f"Error processing file {train_set}: {e}")

# Initialize Random Forest Classifier
classifier = RandomForestClassifier(n_estimators=100, random_state=42, n_jobs=-1)

# Training the Random Forest model on the data
for train_set in tqdm(training_sets, desc="Training Model"):
    file_path = os.path.join(DATASET_DIRECTORY, train_set)
    try:
        for chunk in pd.read_csv(file_path, usecols=X_columns + [y_column], chunksize=5000):  # Include y_column
            X_train = scaler.transform(chunk[X_columns])
            y_train = chunk[y_column]
            classifier.fit(X_train, y_train)
    except Exception as e:
        print(f"Error training on file {train_set}: {e}")

# Test the model on the test sets
for test_set in tqdm(test_sets, desc="Testing Model"):
    file_path = os.path.join(DATASET_DIRECTORY, test_set)
    try:
        for chunk in pd.read_csv(file_path, usecols=X_columns + [y_column], chunksize=5000):  # Include y_column
            X_test = scaler.transform(chunk[X_columns])
            y_test = chunk[y_column]
            predictions = classifier.predict(X_test)
            # Evaluate or log predictions here (e.g., accuracy metrics)
    except Exception as e:
        print(f"Error testing on file {test_set}: {e}")


Mounted at /content/drive


Testing Model: 100%|██████████| 34/34 [04:36<00:00,  8.14s/it]


## Random Forest: 34 Class (33+1 Classification)

In [ ]:
import os
from tqdm import tqdm
from sklearn.ensemble import RandomForestClassifier

# Define the Random Forest model
ML_models = [
    RandomForestClassifier(n_estimators=100, random_state=42, n_jobs=-1),
]

# Assign the model name
ML_names = [
    "RandomForest",
]

# Iterate through each training set
for train_set in tqdm(training_sets, desc="Training Models"):
    # Ensure proper file path concatenation
    file_path = os.path.join(DATASET_DIRECTORY, train_set)
    try:
        # Load the dataset
        d = pd.read_csv(file_path)

        # Apply scaling transformation to the features
        d[X_columns] = scaler.transform(d[X_columns])

        # Train the Random Forest model
        for model, model_name in zip(ML_models, ML_names):
            print(f"Training {model_name} on {train_set}...")
            model.fit(d[X_columns], d[y_column])

        # Clear the data from memory
        del d
    except Exception as e:
        print(f"Error processing {train_set}: {e}")


Training Models:   0%|          | 0/135 [00:00<?, ?it/s]

Training RandomForest on part-00000-363d1ba3-8ab5-4f96-bc25-4d5862db7cb9-c000.csv...


Training Models:   1%|          | 1/135 [00:39<1:27:41, 39.26s/it]

Training RandomForest on part-00001-363d1ba3-8ab5-4f96-bc25-4d5862db7cb9-c000.csv...


Training Models:   1%|▏         | 2/135 [01:13<1:20:41, 36.40s/it]

...

Training Models:  99%|█████████▊| 133/135 [1:40:26<01:45, 52.62s/it]

Training RandomForest on part-00133-363d1ba3-8ab5-4f96-bc25-4d5862db7cb9-c000.csv...


Training Models:  99%|█████████▉| 134/135 [1:41:04<00:48, 48.18s/it]

Training RandomForest on part-00134-363d1ba3-8ab5-4f96-bc25-4d5862db7cb9-c000.csv...


Training Models: 100%|██████████| 135/135 [1:41:45<00:00, 45.22s/it]


In [ ]:
import os
from tqdm import tqdm
import pandas as pd
from sklearn.ensemble import RandomForestClassifier

# Define Random Forest model and its name
ML_models = [
    RandomForestClassifier(n_estimators=100, random_state=42, n_jobs=-1),
]

ML_names = [
    "RandomForest",
]

# Train the Random Forest model on each training set
for train_set in tqdm(training_sets, desc="Training Random Forest"):
    file_path = os.path.join(DATASET_DIRECTORY, train_set)
    try:
        d_train = pd.read_csv(file_path)

        # Apply scaling transformation
        d_train[X_columns] = scaler.transform(d_train[X_columns])

        # Fit the Random Forest model
        for model, model_name in zip(ML_models, ML_names):
            print(f"Training {model_name} on {train_set}...")
            model.fit(d_train[X_columns], d_train[y_column])

        # Clear the training data from memory
        del d_train
    except Exception as e:
        print(f"Error processing {train_set}: {e}")

# Initialize y_test and preds to collect predictions
y_test = []
preds = {i: [] for i in range(len(ML_models))}

# Evaluate the Random Forest model on each test set
for test_set in tqdm(test_sets, desc="Evaluating Random Forest"):
    file_path = os.path.join(DATASET_DIRECTORY, test_set)
    try:
        d_test = pd.read_csv(file_path)

        # Apply scaling transformation
        d_test[X_columns] = scaler.transform(d_test[X_columns])

        # Collect true labels for y_test
        y_test += list(d_test[y_column].values)

        # Predict with the Random Forest model and accumulate predictions
        for i, model in enumerate(ML_models):
            y_pred = list(model.predict(d_test[X_columns]))
            preds[i].extend(y_pred)  # Extend list for each model with new predictions

        # Clear the test data from memory
        del d_test
    except Exception as e:
        print(f"Error processing {test_set}: {e}")

# Now y_test contains true labels, and preds holds predictions for the Random Forest model


Training Random Forest:   0%|          | 0/135 [00:00<?, ?it/s]

Training RandomForest on part-00000-363d1ba3-8ab5-4f96-bc25-4d5862db7cb9-c000.csv...


Training Random Forest:   1%|          | 1/135 [00:37<1:24:36, 37.88s/it]

Training RandomForest on part-00001-363d1ba3-8ab5-4f96-bc25-4d5862db7cb9-c000.csv...


Training Random Forest:   1%|▏         | 2/135 [01:14<1:21:44, 36.88s/it]

...

Training Random Forest:  99%|█████████▊| 133/135 [1:39:43<01:42, 51.16s/it]

Training RandomForest on part-00133-363d1ba3-8ab5-4f96-bc25-4d5862db7cb9-c000.csv...


Training Random Forest:  99%|█████████▉| 134/135 [1:40:21<00:47, 47.23s/it]

Training RandomForest on part-00134-363d1ba3-8ab5-4f96-bc25-4d5862db7cb9-c000.csv...


Evaluating Random Forest: 100%|██████████| 34/34 [04:49<00:00,  8.52s/it]


In [ ]:
# Evaluation phase
from sklearn.metrics import accuracy_score, recall_score, precision_score, f1_score

for k, v in preds.items():
    y_pred = v
    print(f"##### {ML_names[k]} #####")
    print('Accuracy Score: ', accuracy_score(y_test, y_pred))
    print('Recall Score: ', recall_score(y_test, y_pred, average='macro'))
    print('Precision Score: ', precision_score(y_test, y_pred, average='macro'))
    print('F1 Score: ', f1_score(y_test, y_pred, average='macro'))
    print("\n")


##### RandomForest #####
Accuracy Score:  0.9914299206753163
Recall Score:  0.7004519929366664
Precision Score:  0.8085019266522151
F1 Score:  0.7088549720620101




## Random Forest 8 class (7+1) classification

In [ ]:
dict_7classes = {}
dict_7classes['DDoS-RSTFINFlood'] = 'DDoS'
dict_7classes['DDoS-PSHACK_Flood'] = 'DDoS'
dict_7classes['DDoS-SYN_Flood'] = 'DDoS'
dict_7classes['DDoS-UDP_Flood'] = 'DDoS'
dict_7classes['DDoS-TCP_Flood'] = 'DDoS'
dict_7classes['DDoS-ICMP_Flood'] = 'DDoS'
dict_7classes['DDoS-SynonymousIP_Flood'] = 'DDoS'
dict_7classes['DDoS-ACK_Fragmentation'] = 'DDoS'
dict_7classes['DDoS-UDP_Fragmentation'] = 'DDoS'
dict_7classes['DDoS-ICMP_Fragmentation'] = 'DDoS'
dict_7classes['DDoS-SlowLoris'] = 'DDoS'
dict_7classes['DDoS-HTTP_Flood'] = 'DDoS'

dict_7classes['DoS-UDP_Flood'] = 'DoS'
dict_7classes['DoS-SYN_Flood'] = 'DoS'
dict_7classes['DoS-TCP_Flood'] = 'DoS'
dict_7classes['DoS-HTTP_Flood'] = 'DoS'


dict_7classes['Mirai-greeth_flood'] = 'Mirai'
dict_7classes['Mirai-greip_flood'] = 'Mirai'
dict_7classes['Mirai-udpplain'] = 'Mirai'

dict_7classes['Recon-PingSweep'] = 'Recon'
dict_7classes['Recon-OSScan'] = 'Recon'
dict_7classes['Recon-PortScan'] = 'Recon'
dict_7classes['VulnerabilityScan'] = 'Recon'
dict_7classes['Recon-HostDiscovery'] = 'Recon'

dict_7classes['DNS_Spoofing'] = 'Spoofing'
dict_7classes['MITM-ArpSpoofing'] = 'Spoofing'

dict_7classes['BenignTraffic'] = 'Benign'

dict_7classes['BrowserHijacking'] = 'Web'
dict_7classes['Backdoor_Malware'] = 'Web'
dict_7classes['XSS'] = 'Web'
dict_7classes['Uploading_Attack'] = 'Web'
dict_7classes['SqlInjection'] = 'Web'
dict_7classes['CommandInjection'] = 'Web'


dict_7classes['DictionaryBruteForce'] = 'BruteForce'